# Test null map generation functions

In [1]:
import sys
import os
import pathlib 
import numpy as np
import pandas as pd
from IPython.display import display
from neuromaps.images import load_nifti, load_gifti

# current path
wd = pathlib.Path().resolve().parent
print(wd)

# import juspyce
sys.path.append(os.path.dirname(os.path.join(wd, "juspyce")))
from juspyce.api import JuSpyce

/Users/llotter/projects/juspyce


## Get parcellations

In [2]:
# volumetric
parc_vol = load_nifti(os.path.join(wd, "testing", "test_parc", "Schaefer100-7_TianS1_2mm.nii.gz"))
n_parcels_vol = len(np.trim_zeros(np.unique(parc_vol.get_fdata())))
print(n_parcels_vol)

# surface both hemis
parc_surf = (load_gifti(os.path.join(wd, "testing", "test_parc", "destrieux.fsa.10k.L.label.gii")),
             load_gifti(os.path.join(wd, "testing", "test_parc", "destrieux.fsa.10k.R.label.gii")))
n_parcels_surf = (len(np.trim_zeros(np.unique(parc_surf[0].darrays[0].data))),
                  len(np.trim_zeros(np.unique(parc_surf[1].darrays[0].data))))
print(n_parcels_surf)

# surface left hemi
parc_surf_lh = parc_surf[0]
n_parcels_surf_lh = n_parcels_surf[0]
print(n_parcels_surf_lh)

116
(74, 74)
74


## Test distance matrix calculation

In [3]:
from juspyce.nulls import get_distance_matrix

# volumetric
print("Volumetric")
dist = get_distance_matrix(parc_vol, "MNI152")
print(type(dist), len(dist))
display(pd.DataFrame(dist))

# surface both hemis
print("Surface - both")
dist = get_distance_matrix(parc_surf, "fsaverage", parc_hemi=["L", "R"], n_cores=8)
print(type(dist), len(dist))
display(pd.DataFrame(dist[0]))
display(pd.DataFrame(dist[1]))

# surface one hemi
print("Surface - left")
dist = get_distance_matrix(parc_surf_lh, "fsaverage", parc_hemi=["L"])
print(type(dist), len(dist))
display(pd.DataFrame(dist))

Volumetric


Calculating distance matrix:   0%|          | 0/116 [00:00<?, ?it/s]

<class 'numpy.ndarray'> 116


,0,1,2,3,4,5,6,7,8,9,...,106,107,108,109,110,111,112,113,114,115
0,0.000000,46.026604,33.721268,65.145287,65.707100,43.989674,53.251812,68.881470,68.916901,42.830582,...,66.535881,66.289368,19.624897,31.298292,27.342598,39.001465,52.012741,35.743858,40.444614,55.000160
1,46.026604,0.000000,25.400406,26.864336,30.750854,31.447741,35.893322,40.199345,46.549068,66.543327,...,95.429375,99.582245,56.178623,74.047806,55.014908,69.409180,93.130302,74.907661,79.371223,92.382347
2,33.721268,25.400406,0.000000,39.798676,35.495369,22.355152,39.674416,43.678761,42.380585,57.434650,...,77.080132,80.795021,42.278198,59.623802,37.336876,51.280720,76.184692,58.256878,63.544510,74.812157
3,65.145287,26.864336,39.798676,0.000000,27.394293,39.231438,37.701271,32.039948,42.510342,79.706429,...,110.774406,115.125069,74.796707,93.203796,71.124176,84.850189,111.045380,92.392075,96.459793,108.571716
4,65.707100,30.750854,35.495369,27.394293,0.000000,31.985382,50.473072,35.107979,33.330921,86.550995,...,98.874741,106.031105,74.691216,92.009216,66.761063,79.218971,107.026978,89.799530,95.423447,104.458740
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111,39.001465,69.409180,51.280720,84.850189,79.218971,52.453995,69.468933,78.167923,70.132301,48.834446,...,41.230312,35.409554,33.553623,34.256222,18.331335,0.000000,33.381077,21.054539,27.783798,27.983427
112,52.012741,93.130302,76.184692,111.045380,107.026978,81.020866,94.426445,107.634491,101.332047,59.429459,...,42.061672,30.843819,42.045494,26.613182,42.742783,33.381077,0.000000,21.064293,23.247938,18.319929
113,35.743858,74.907661,58.256878,92.392075,89.799530,63.043461,74.635872,88.701996,83.864815,43.461880,...,46.984619,38.468670,26.968555,19.509529,25.321857,21.054539,21.064293,0.000000,15.420214,22.352907
114,40.444614,79.371223,63.544510,96.459793,95.423447,68.219154,76.098236,92.005592,88.366791,40.704655,...,54.027538,43.424641,31.500271,23.867083,31.801302,27.783798,23.247938,15.420214,0.000000,23.385948


Surface - both


...

,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,0.000000,208.245743,149.784134,110.620148,23.910089,58.255966,93.452415,121.412811,149.737930,171.214767,...,197.538376,119.029366,164.565201,88.298576,109.548790,41.578789,162.823181,175.712036,174.350937,132.535202
1,208.245728,0.000000,152.966125,136.790894,225.480530,202.488937,170.942978,148.958191,115.714516,84.080803,...,98.875153,139.780731,113.679337,176.649094,191.149475,212.387802,118.992096,58.509995,67.010475,111.530670
2,149.784164,152.966171,0.000000,114.302185,138.594559,109.730431,68.009628,46.579800,62.012745,91.012428,...,86.749947,75.406181,62.003262,103.373230,56.145493,136.001755,55.025394,168.780487,130.683502,141.110229
3,110.620125,136.790924,114.302155,0.000000,121.514252,149.380539,152.465912,148.493408,151.670670,158.873932,...,140.955917,162.541107,61.946716,47.341595,84.267891,143.038300,137.532516,126.097702,89.650955,54.761578
4,23.910095,225.480560,138.594559,121.514259,0.000000,46.914459,77.443962,105.324905,134.505203,162.187378,...,184.247284,104.517891,168.040451,90.951935,103.951485,41.601532,147.396637,193.437149,189.978729,149.190720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,41.578793,212.387833,136.001740,143.038269,41.601543,30.260578,69.840439,96.022774,118.103813,144.858032,...,167.114990,86.391289,184.668930,126.866440,134.208282,0.000000,130.967316,180.053391,180.785309,137.833633
70,162.823212,118.992050,55.025406,137.532516,147.396606,109.645485,72.297684,47.060551,21.908751,39.673679,...,45.027458,50.503101,78.212517,152.270981,106.368469,130.967316,0.000000,141.347321,132.800781,152.645706
71,175.712051,58.509983,168.780502,126.097733,193.437149,192.956772,190.996262,170.259811,136.975677,105.122391,...,128.994888,157.819504,117.914566,164.601456,192.405212,180.053421,141.347305,0.000000,54.201012,86.276665
72,174.350845,67.010460,130.683502,89.650970,189.978790,191.140793,186.481461,165.898651,145.756668,132.358994,...,120.083397,168.394806,78.368988,131.092743,153.227890,180.785370,132.800827,54.201015,0.000000,55.627800


Surface - left


Calculating distance matrix (1 proc):   0%|          | 0/1 [00:00<?, ?it/s]

<class 'numpy.ndarray'> 74


,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,0.000000,211.429443,154.179047,109.072639,22.115067,60.872902,95.328476,122.768150,149.891830,167.550980,...,196.721085,120.811409,163.953751,86.489319,114.387474,53.448647,163.367706,171.759460,172.973969,135.186584
1,211.429474,0.000000,156.397629,139.500946,225.757843,199.772598,167.225372,146.787964,112.707344,82.018311,...,92.546120,136.113708,116.288727,178.006897,193.200165,206.901398,118.388657,61.197079,65.917404,108.562019
2,154.179001,156.397675,0.000000,112.806564,142.614120,109.816689,67.599884,46.310341,61.693611,89.783989,...,89.834465,72.360794,66.845703,105.865028,55.939178,132.738022,55.199200,174.232193,142.584839,148.954498
3,109.072647,139.500977,112.806549,0.000000,124.625275,152.942947,154.270660,149.126144,151.170639,154.428116,...,141.743668,161.478058,59.663933,46.225163,84.224144,138.553574,135.100616,125.067001,92.135033,56.861050
4,22.115070,225.757812,142.614075,124.625259,0.000000,45.189667,78.526886,106.097130,133.091675,159.012878,...,182.523087,104.220467,175.056290,98.064056,115.897255,43.987240,146.682205,186.992599,188.527649,150.647385
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,53.448647,206.901413,132.737976,138.553589,43.987240,29.649273,67.375885,92.439812,114.227928,140.042267,...,163.569046,84.135139,180.325500,133.750870,136.259888,0.000000,128.029556,167.713043,171.429108,132.375671
70,163.367706,118.388649,55.199181,135.100616,146.682236,108.772125,71.765236,46.779861,22.650221,39.481663,...,45.027805,49.401230,78.875397,154.438950,106.986938,128.029556,0.000000,137.531570,136.650757,159.976883
71,171.759430,61.197098,174.232239,125.066986,186.992615,185.858521,184.084824,164.209930,130.153992,99.002678,...,124.923073,150.331284,121.754097,163.818130,192.978958,167.713058,137.531555,0.000000,53.856354,79.059319
72,172.974075,65.917381,142.584793,92.135002,188.527679,187.880997,188.892960,168.765427,146.804764,127.599762,...,123.758080,166.196945,85.074226,132.509842,157.306778,171.429123,136.650742,53.856354,0.000000,54.355080


## Test null maps generation incl distance matrices

In [3]:
from juspyce.nulls import generate_null_maps

# volumetric
print("Volumetric")
null_maps, dist = generate_null_maps(
    data=np.random.random((10,n_parcels_vol)), 
    parcellation=parc_vol,
    parc_space="MNI152",
    n_nulls=1000,
    n_cores=-1)
print(type(null_maps), len(null_maps), null_maps[0].shape)
display(pd.DataFrame(null_maps[0]))

# surface both hemis
print("Surface - both")
null_maps, dist = generate_null_maps(
    data=np.random.random((10,np.sum(n_parcels_surf))), 
    parcellation=parc_surf,
    parc_space="fsaverage",
    parc_hemi=["L", "R"],
    parc_density="10k",
    n_nulls=1000,
    n_cores=-1)
print(type(null_maps), len(null_maps), null_maps[0].shape)
display(pd.DataFrame(null_maps[0]))

# surface one hemi
print("Surface - left")
null_maps, dist = generate_null_maps(
    data=np.random.random((10,n_parcels_surf_lh)), 
    parcellation=parc_surf_lh,
    parc_space="fsaverage",
    parc_hemi=["L"],
    parc_density="10k",
    n_nulls=1000,
    n_cores=-1)
print(type(null_maps), len(null_maps), null_maps[0].shape)
display(pd.DataFrame(null_maps[0]))

INFO:juspyce.nulls:Null map generation: Assuming n = 10 data vector(s) for n = 116 parcels.
INFO:juspyce.nulls:Loaded parcellation (parc_space = 'MNI152').
INFO:juspyce.nulls:Calculating distance matrix/matrices (space = 'MNI152').


Volumetric


Generating null maps (-1 proc):   0%|          | 0/10 [00:00<?, ?it/s]

INFO:juspyce.nulls:Null data generation finished.


<class 'dict'> 10 (1000, 116)


...

Surface - left


Generating null maps (-1 proc):   0%|          | 0/10 [00:00<?, ?it/s]

INFO:juspyce.nulls:Null data generation finished.


<class 'dict'> 10 (1000, 74)


,0,1,2,3,4,5,6,7,8,9,...,64,65,66,67,68,69,70,71,72,73
0,-0.275854,-0.765660,0.095356,-0.210737,-0.174199,-0.066561,0.183275,0.285071,-0.264033,0.101150,...,0.291285,0.037595,-0.119373,0.378390,0.318912,-0.036585,0.069041,-0.222457,-0.479025,-0.440534
1,0.012773,-0.412908,0.544031,-0.174025,0.127506,-0.118008,0.884472,0.624788,0.283260,-0.018869,...,-0.566266,-0.223863,-0.116075,0.364955,0.285275,0.362336,-0.446722,0.227933,0.045725,-0.095402
2,0.012191,-0.021356,-0.002796,-0.250411,0.538911,-0.254823,-0.132692,0.212173,0.199852,0.095992,...,0.006108,-0.074459,-0.172287,-0.012294,0.406490,0.301248,0.068077,-0.267329,-0.242997,0.366748
3,0.728553,-1.097213,1.132631,-0.447672,0.131698,0.577473,0.833707,-0.316207,-0.052856,0.537360,...,-0.444632,-0.234463,0.274822,-0.041904,-1.497461,0.601216,-0.086294,0.045720,-0.304923,0.406586
4,0.043983,-0.319137,-0.487069,-0.644949,0.405264,-0.115305,-0.110960,0.353439,-0.438633,0.076372,...,-0.451524,0.151395,-0.087991,0.034897,-0.485468,0.952497,-0.060994,-0.070517,-0.411171,0.145533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.090529,0.346688,0.179599,0.051123,-0.031663,-0.330517,-0.171543,-0.303128,0.502154,-0.917179,...,-0.363880,-0.282078,-0.246103,-0.126170,0.146361,0.238704,-0.071930,0.088725,-0.235945,-0.029701
996,0.012247,-0.009979,-0.188853,0.135209,-0.309737,0.829706,-0.408048,-0.252218,-0.326731,-0.062046,...,-0.485104,-0.388533,0.171654,-0.225778,-0.344673,-0.010521,-0.741995,0.115324,0.593271,-0.241613
997,-0.447124,-0.063259,0.429992,0.643719,0.173255,0.035742,0.071256,-0.302928,-0.087011,-0.601783,...,-0.172458,-0.664832,0.314952,0.063913,0.340801,0.135990,0.226071,-0.280841,0.654275,0.070780
998,0.203596,-0.232651,-0.286386,-0.208318,-0.222285,0.711980,-0.325251,-0.187096,-0.489419,-0.597784,...,0.141967,-0.339965,-0.099996,0.006723,-0.586460,0.048324,-0.766158,0.045451,-0.160196,0.029358
